# VTC MyPortal Calendar Export

!!! Warning !!!

Do NOT `Run All`. Do NOT `Run All`. Do NOT `Run All`. 

It is garenteed to screw up when `Run All`.

run the blocks one by one.

In [ ]:
!pip install -r requirements.txt

## initializations

1. create folder data
2. define variables

In [ ]:
import os
DATA_DIR = "data"

if not os.path.exists(DATA_DIR):
    os.mkdir(DATA_DIR)

MYPORTAL_BASE_URL = 'https://myportal.vtc.edu.hk'
CALENDAR_START_DATE = '20240900'
CALENDAR_END_DATE = '20250500'
CALENDAR_EVENTS_FILE = 'calendar_events.json'
CALENDAR_EXPORT_FILE = 'calendar_export.ics'

CALENDAR_EVENTS_FILE = os.path.join(DATA_DIR, CALENDAR_EVENTS_FILE)
CALENDAR_EXPORT_FILE = os.path.join(DATA_DIR, CALENDAR_EXPORT_FILE)

## Open Broswer for login

after runing the block, a browser will open, login to the VTC MyPortal before continuing.

In [ ]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()

In [ ]:
from selenium.webdriver import Chrome

chrome_broswer = Chrome(options=chrome_options)
chrome_broswer.get(MYPORTAL_BASE_URL)

## Open Calander and Extract Data

Only Continue after login.

below steps are only needed to fetch feed. if data already present in data folder skip to [Open Calander File](#-Open-Calander-File)

In [ ]:
from selenium.webdriver.common.by import By

chrome_broswer.find_element(By.LINK_TEXT, 'Calendar').click()

In [ ]:
# Get Feed URL
from urllib.parse import urljoin

calender_feed_url = chrome_broswer.execute_script("return eventFeedUrl;")
calender_feed_url = urljoin(MYPORTAL_BASE_URL, calender_feed_url)
calender_feed_url = f'{calender_feed_url}&from={
    CALENDAR_START_DATE}&to={CALENDAR_END_DATE}'

In [ ]:
# open feed
chrome_broswer.get(calender_feed_url)

In [ ]:
# Extract json
import json

calendar_events = chrome_broswer.find_element(By.TAG_NAME, 'pre').text
calendar_events = json.loads(calendar_events)
calendar_events

In [ ]:
# fileter
calendar_events = list(
            filter(lambda e: e['eventType'] != 'Holiday', calendar_events))
calendar_events = list(
            filter(lambda e: e['eventType'] != 'InstitutionHoliday', calendar_events))
calendar_events

## Save Exported data

save to data/

In [ ]:
with open(CALENDAR_EVENTS_FILE, 'w') as f:
    json.dump(calendar_events, f, indent=4)

## Open Calander File

In [ ]:
calendar_events = json.load(open(CALENDAR_EVENTS_FILE))
calendar_events

In [ ]:
# Create ical
import pytz
from datetime import datetime
from icalendar import Calendar, Event, vCalAddress, vText

cal = Calendar()

In [ ]:
# Create ical events
for event in calendar_events:

    # adjust time to UTC
    local = pytz.timezone("Asia/Hong_Kong")
    start = datetime.strptime(
        event['startDateTime'], "%Y%m%dT%H%M%S")
    start = local.localize(start, is_dst=None).astimezone(pytz.utc)
    end = datetime.strptime(event['endDateTime'], "%Y%m%dT%H%M%S")
    end = local.localize(end, is_dst=None).astimezone(pytz.utc)

    ical_event = Event()
    # add event details
    ical_event.add('summary', f'{event["summary"]} <BOT>')
    ical_event.add('description',
                   f"{event['details']}\n\nLast updated by MYPORTAL_iCALENDER_BOT@{datetime.now()}")
    ical_event.add('categories', event['eventType'])
    ical_event.add('dtstart', start)
    ical_event.add('dtend', end)

    # add bot as the organizer
    organizer = vCalAddress('MAILTO:VTC_MYPORTAL_iCAL_BOT@example.com')
    organizer.params['cn'] = vText('MYPORTAL_iCAL_BOT')
    organizer.params['role'] = vText('BOT')
    ical_event['organizer'] = organizer

    # add location of the event
    ical_event['location'] = vText(event['location'])

    cal.add_component(ical_event)

In [ ]:
with open(CALENDAR_EXPORT_FILE, 'wb') as f:
    f.write(cal.to_ical())